In [1]:
from TrainedVAE import TrainedLinearVAE2
from TrainedVAE import TrainedConvVAE2
import torch
import torch.distributions as tdist
import dataloader

In [2]:
_, test_loader = dataloader.setup_data_loaders(batch_size=60000, use_cuda=False)
X, classes = next(iter(test_loader)) 

In [12]:
model = TrainedLinearVAE2
SD = torch.tensor([0.05])
z_mean, z_scale = model.encoder(X)
q_z_x = tdist.Normal(z_mean, z_scale)
Z = q_z_x.sample()
Z_param = model.decoder(Z)
p_x_z = tdist.Normal(loc = Z_param, scale=SD)
p_z = tdist.MultivariateNormal(loc=torch.zeros(2), covariance_matrix=torch.eye(2))

In [20]:
with torch.no_grad():
    log_p_x = p_x_z.log_prob(X.view(-1, 784)).sum()
    log_p_z = p_z.log_prob(Z).sum()
    log_q_z = q_z_x.log_prob(Z).sum()
    
    out = torch.exp(log_p_x + log_p_z) / torch.exp(log_q_z)

In [24]:
model = TrainedLinearVAE2
SD = torch.tensor([0.05])
p_z = tdist.MultivariateNormal(loc=torch.zeros(2), covariance_matrix=torch.eye(2))
Z = p_z.sample((10000,))
Z_param = model.decoder(Z)
p_x_z = tdist.Normal(loc = Z_param, scale=SD)

tensor([[ 0.3867, -0.2701],
        [ 0.8027, -0.5531],
        [-0.8940,  0.4052],
        ...,
        [-0.6915,  0.9183],
        [-1.9317, -0.9284],
        [-0.0251,  0.4767]])

In [ ]:
with torch.no_grad():
    S = 0
    for idx, z in enumerate(Z_param):
        p_x_z = tdist.Normal(loc = z, scale=SD)
        S += torch.exp(p_x_z.log_prob(X.view(-1, 784)).sum())

torch.Size([10000, 2])